In [1]:

# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *


class DeepConvNet:
    """정확도 99% 이상의 고정밀 합성곱 신경망

    네트워크 구성은 아래와 같음
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화===========
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가（TODO: 자동 계산되게 바꿀 것）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]
            


In [ ]:
            # coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()  
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=20, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr':0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보관
network.save_params("deep_convnet_params.pkl")
print("Saved Network Parameters!")

train loss:2.2935973693278533
=== epoch:1, train acc:0.121, test acc:0.108 ===
train loss:2.2921287813192617
train loss:2.2765483610202075
train loss:2.2934699043294553
train loss:2.294995093089073
train loss:2.253465435053731
train loss:2.2975309355413485
train loss:2.2937368493124946
train loss:2.2670947225481934
train loss:2.2621722951851924
train loss:2.2440485023349894
train loss:2.2208055169277308
train loss:2.2466226918840677
train loss:2.2203284878437173
train loss:2.217710079496211
train loss:2.1954819000538914
train loss:2.254555767650513
train loss:2.192551038286831
train loss:2.22958311711636
train loss:2.1478657413125517
train loss:2.1953477474061263
train loss:2.1303943512476113
train loss:2.1266079879281863
train loss:2.1096416016107997
train loss:2.096335909362765
train loss:2.1678232106132467
train loss:2.129768390925228
train loss:2.144999943181999
train loss:2.063915182380604
train loss:1.927388214653635
train loss:2.079129869057731
train loss:2.0850727259229314
trai

train loss:1.4914415367286755
train loss:1.5203845275028756
train loss:1.3609230238522003
train loss:1.252672512400224
train loss:1.317024931521861
train loss:1.2998432837774812
train loss:1.172534493810737
train loss:1.183160190958145
train loss:1.2374101496118777
train loss:1.3456159500303975
train loss:1.386438672851817
train loss:1.438939855354978
train loss:1.357239477223739
train loss:1.249863904094446
train loss:1.2707960307438062
train loss:1.2634392446572067
train loss:1.2254754542562418
train loss:1.2066902107521542
train loss:1.3255634727118655
train loss:1.1123776724337444
train loss:1.2096256637915699
train loss:1.0557378763027128
train loss:1.3412137627496272
train loss:1.4332436360787582
train loss:1.5898150715514117
train loss:1.3791361450589505
train loss:1.2385137717881094
train loss:1.3821419233261156
train loss:1.2280765595903962
train loss:1.5994158240733696
train loss:1.4019749518528908
train loss:1.3868506525948758
train loss:1.1221835755024376
train loss:1.26801

train loss:1.2141502466971845
train loss:1.1364399283406996
train loss:1.2063501614429075
train loss:1.1083554407876381
train loss:1.205663721388112
train loss:1.1811788811202184
train loss:1.24934765178749
train loss:1.060238102335024
train loss:1.0931806550957335
train loss:1.0100133803415519
train loss:1.2080310651692654
train loss:1.1401429898283317
train loss:0.9841611019622996
train loss:1.034093752482911
train loss:1.1123599755169689
train loss:1.1248941819576908
train loss:0.9126229145473165
train loss:1.1126457806537375
train loss:1.2377852012697481
train loss:1.1490515244140398
train loss:0.9244701830586165
train loss:1.1738343162163476
train loss:0.9482298364033581
train loss:1.2683375294659376
train loss:1.0795463629795952
train loss:1.1923577278372377
train loss:0.9406548652335524
train loss:1.0942924301250077
train loss:1.2512101450209614
train loss:1.1223609192954793
train loss:1.057535332640685
train loss:1.1479425998664659
train loss:1.0802458025671084
train loss:1.041

train loss:1.0877541622617546
train loss:0.948107000735244
train loss:1.0438941731398643
train loss:1.2701735906122076
train loss:1.0850044024514482
train loss:1.0738142327190983
train loss:1.0385941536217036
train loss:1.1158798931846783
train loss:1.2701136589779327
train loss:1.0331536183071248
train loss:1.0863601726003562
train loss:0.9583512685479135
train loss:0.9643912085552881
train loss:1.287829106315935
train loss:0.9297076717618374
train loss:1.0993071997371837
train loss:1.1610180958478107
train loss:1.0050167119969773
train loss:0.9940926814564759
train loss:0.9976889617703018
train loss:1.033247209254894
train loss:1.1714551610249515
train loss:1.0477027680524897
train loss:1.1735614012394457
train loss:1.149315119328651
train loss:1.1603757300847455
train loss:1.2510902533429107
train loss:0.9832801927026117
train loss:1.0429695100755436
train loss:1.0829547809377704
train loss:1.2012942760475371
train loss:1.0389441087074227
train loss:1.030100460118141
train loss:0.94

train loss:0.9913842672126332
train loss:1.067975143694255
train loss:1.025562878562088
train loss:0.9496886381233597
train loss:0.9018434369800937
train loss:0.9315719938646967
train loss:1.0174783184575107
train loss:0.9283473637735713
train loss:0.9973117023984319
train loss:1.134120544628933
train loss:1.0378766454189943
train loss:1.0200139007496896
train loss:1.12956728815769
train loss:1.0666241312445102
train loss:0.93191580562943
train loss:0.866753744520761
train loss:0.9994174819120919
train loss:1.143478202751445
train loss:0.9490651881857107
train loss:0.9762836152880383
train loss:0.9406268708644512
train loss:0.9297696257276459
train loss:1.011668555802167
train loss:1.1612524988745825
train loss:1.1154206637486843
train loss:0.909738764572213
train loss:0.9091867672968212
train loss:1.0085260552061694
train loss:0.9342994827331143
train loss:0.9134782955116154
train loss:0.9615393554973309
train loss:0.9609853937271992
train loss:1.1250995142295919
train loss:1.01890638

...

train loss:0.9371606459069883
train loss:0.8894609652790814
train loss:0.8423765907768674
train loss:0.815081178359288
train loss:0.9087547163806782
train loss:1.068770548468797
train loss:0.947167154606117
train loss:0.8795711446604012
train loss:0.8190892250061019
train loss:0.9150642436458122
train loss:1.045414158621415
train loss:1.0413001549742034
train loss:0.9960359675499383
train loss:0.9998834423514499
train loss:0.8706287251569
train loss:0.9476147779148318
train loss:0.9731571186228115
train loss:0.8588913968548458
train loss:0.8119686133985351
train loss:0.912153427707363
train loss:1.081667266079168
train loss:0.8260881095497138
train loss:0.8991354377553705
train loss:0.884167117166142
train loss:0.7582962292788353
train loss:0.9796445995225388
train loss:0.9204817593329547
train loss:1.009443513186254
train loss:0.8916187088643145
train loss:0.9223282161165396
train loss:0.7232953997570211
train loss:1.0067326417532991
train loss:0.8250482846192042
train loss:0.81693679

train loss:0.9936639408311025
train loss:0.9675671077344912
train loss:0.7784801997036205
train loss:0.8520587952035811
train loss:0.9579069851284417
train loss:1.1113300536815118
train loss:0.8462293369858587
train loss:0.7719123798330565
train loss:0.9123450116749803
train loss:0.9080548201061245
train loss:0.8765528785534491
train loss:0.7603481025905797
train loss:0.9399526133705521
train loss:0.7773640300320577
train loss:0.85796512059636
train loss:0.819531526364359
train loss:0.9315414087373918
train loss:0.914692244177217
train loss:0.8988705877302355
train loss:0.8744206116329423
train loss:0.8430766122713912
train loss:0.8107257320408381
train loss:0.7808776041401545
train loss:0.7421222370634982
train loss:1.0224989314581714
train loss:0.8605796383353291
train loss:0.9840899692671407
train loss:0.8478345224787386
train loss:1.0767190581393162
train loss:0.9022279513470288
train loss:0.8581929281535962
train loss:1.0745224858411067
train loss:0.8173227131032518
train loss:0.9

train loss:0.8101288783667009
train loss:0.7682405665427865
train loss:0.8133797935923522
train loss:0.8351270075184333
train loss:0.9462710693225947
train loss:0.9431802540045865
train loss:0.9366654764155029
train loss:0.808436821456649
train loss:1.0306101516181236
train loss:0.9977117701870735
train loss:0.9997190144761984
train loss:0.7544597077502122
train loss:0.8385751070935044
train loss:0.8547700618147186
train loss:0.8920496117937582
train loss:0.8893268050760597
train loss:0.8938306048969682
train loss:0.7719608148424316
train loss:0.9075916809588815
train loss:0.8930436121117696
train loss:0.836957036562507
train loss:0.8234690914981997
train loss:1.056107830659802
train loss:0.8225739557195386
train loss:0.9670786435260093
train loss:0.9104557225313065
train loss:0.8104174502634202
train loss:1.050566880729475
train loss:0.884923804839475
train loss:0.900360993290585
train loss:0.8122864889315806
train loss:0.7864854177566875
train loss:0.8096766408592088
=== epoch:7, tra

train loss:0.8292265291984804
train loss:0.7642799508248248
train loss:1.0338569870961447
train loss:0.8279094654353107
train loss:0.7322235932913401
train loss:0.7817670479848914
train loss:0.945824628503277
train loss:0.8373094824370795
train loss:0.8438990293697153
train loss:0.7549019838608886
train loss:0.8348854832501417
train loss:1.0687926186317152
train loss:0.9367682152790571
train loss:0.9891455538090977
train loss:0.8746823046682518
train loss:0.698062940827601
train loss:1.0139936405434773
train loss:0.7507698222562413
train loss:0.9615571174038547
train loss:0.8004891311306056
train loss:0.8365982401847504
train loss:0.8475417167256389
train loss:0.7509142941139004
train loss:0.870582109974356
train loss:0.8649116985600982
train loss:0.8648075179296972
train loss:0.8621761498670928
train loss:0.9249502330672774
train loss:0.9171566845850784
train loss:0.9402891957831244
train loss:0.8303919916209673
train loss:0.748280413534359
train loss:1.1693248215604175
train loss:0.8

train loss:0.8340784602251792
train loss:1.0030865756562124
train loss:0.8121195072795528
train loss:0.8307701670488338
train loss:0.7206477392280454
train loss:0.7305966191918059
train loss:0.9351847262289441
train loss:0.8988365905336038
train loss:0.9462656118792286
train loss:0.8936841048923018
train loss:0.7619919965558607
train loss:0.8601483407495345
train loss:0.8959708031132914
train loss:0.8462978468779745
train loss:0.8716280837002663
train loss:0.6456386009787097
train loss:0.8811110195549563
train loss:0.8849280173316789
train loss:0.821392097230899
train loss:0.9165898664176464
train loss:1.2419019344240607
train loss:0.8868699502772373
train loss:0.9013521972273862
train loss:0.8981864061332671
train loss:0.8775403829724528
train loss:1.1391572789196514
train loss:0.7637964661795438
train loss:1.248886036189688
train loss:0.8529923704384466
train loss:0.927008236297148
train loss:0.9725890316443581
train loss:0.7755190593857413
train loss:0.7835275144982963
train loss:0.